# NASA FRIMS Fire Data Visualization
---

Dataset retreived from NRT VIIRS 375 m Active Fire product VNP14IMGT distributed from NASA FIRMS.
>Please see `NASA-FIRMS_fetch.ipynb` for data scraping and `NASA_FIRMS_merge.ipynb` for data scrubbing


In [89]:
# Dependencies
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colormaps as cmaps
from matplotlib.colors import ListedColormap as lcmap
from matplotlib.colors import Normalize as norm
from matplotlib.colors import LogNorm as logn

## 1 Set Variables and Load Data
---

### 1.1 Set Fixed Notebook Variables and Create Daterange

In [90]:
# global variables
crs = 4326 # coordinate reference system - EPSG:4326 WGS84 - World Geodetic System 1984, used in GPS
# month list as %Y-%m
dt = [f'{str(x.date())}'[0:7] for x in pd.date_range("2013-01-01", "2024-07-09", freq="ME")]

### 1.2 Load Shapes

In [91]:
# Load ca boundary/bounding box/grid shape file
ca_bd = gpd.read_file('../data/shapes/ca_shape.geojson')
ca_bbox = gpd.read_file('../data/shapes/ca_bbox.geojson')
ca_gdf = gpd.read_file('../data/shapes/ca_grid.geojson')

### 1.3 Load Fire Data

In [92]:
# Read NASA FIRMS fire data for year
fire_df = pd.read_csv(f'../data/nasa-firms/all.csv')[['acq_date','latitude','longitude','frp']]

## 2 Parse Data
---

### 2.1 Parse Fire Data

In [97]:
# Create GeoDataframe from Dataframe and add count column
fire_gdf = gpd.GeoDataFrame(fire_df[['acq_date','frp']], geometry = gpd.points_from_xy(fire_df.longitude,fire_df.latitude), crs=crs)
fire_gdf.insert(2,'cnt',1)

In [98]:
fire_gdf.tail(1)

,acq_date,frp,cnt,geometry
918265,2024-07-09,4.64,1,POINT (-123.01070 41.53033)


In [99]:
# sjoin to filter fire data points within ca state boundary
ca_fire_gdf = gpd.sjoin(fire_gdf,ca_gdf, how='left', predicate='within').dropna()

In [100]:
filter = ca_fire_gdf['index_right'].unique().astype('int')

In [15]:
ca_gdf.loc[filter,:].to_file('../data/nasa-firms/geojson/ca_fire_GES_grids.geojson', driver='GeoJSON')

In [16]:
ca_fire_gdf[['acq_date','frp','cnt','index_right']].to_csv('../data/nasa-firms/geojson/ca_fire_GES_data.csv')

### 2.2 Bin Fire Data by Month

In [101]:
# variables to store
ca_fire_dict = {}
ca_gdf_dict ={}
# Loop through months and output fire plots
for d in dt:
    # Group Fire by month
    fire_mo_gdf = ca_fire_gdf.loc[ca_fire_gdf['acq_date'].str[0:7] == d][['index_right','frp','cnt','geometry']] #length is ttl incidents in set
    # Dissolve on sum of values
    dissolve = fire_mo_gdf.dissolve(by='index_right', aggfunc='sum') #grouped points in active fire grids of incidents in ca
    # Store ca geodataframe to dict
    ca_fire_dict[d] = dissolve
    ca_gdf_dict[d] = gpd.GeoDataFrame({'grid_index':[int(x) for x in dissolve.index], 'frp':dissolve.frp.values, 'cnt':dissolve.cnt.values},
                         geometry = ca_gdf.geometry[dissolve.index].to_list(), crs=crs)

## 2 Plot Data
---

In [106]:
# set custom colomap
afmhot = cmaps['afmhot']
afmhot2 = lcmap(afmhot(np.linspace(0.1, 0.7, 256)))
n1 = logn(vmin=0.07, vmax=52558.2)

In [107]:
# Create Plots
for d in dt:
    for c in ['frp']: #,'cnt','frp/cnt']
        # Plot ca cells
        ax = ca_fire_dict[d].plot(figsize=(11,10), column=c, cmap=afmhot2, alpha=0.8,
                                 markersize=1, vmin=0.07, vmax=52558.2, norm=n1, zorder=3)
        ax.axis('off')
        ca = ca_bd.to_crs(crs=4326).plot(ax=ax, color='#333', zorder=2) # Plot state boundary
        bbox = ca_bbox.to_crs(crs=4326).plot(ax=ax, color='#fff', zorder=1) # Plot bounding box
        plt.colorbar(cm.ScalarMappable(norm=n1, cmap=afmhot2), ax=ax, 
                     location='right', anchor=(0, 0.3), shrink=0.5) # Configure colorbar
        v = ca_gdf_dict[d][c]
        plt.title(f'{d}: min={v.min():.{2}f} median={np.median(v):.{1}f} max={v.max():.{0}f}', loc='left')
        plt.savefig(f'../images/nasa-firms/{c}/{c}_{d}.png')
        plt.close()